# Class, Struct, Record and Enum - Good vs Bad Examples

This notebook demonstrates **meaningful** vs **trivial** implementations of classes, structs, records, and enums.

## Classes

### ✅ Good Example - Meaningful Class Usage

In [ ]:
// ✅ GOOD: User class with validation, business logic, and encapsulation
public class User
{
    private string _email;
    private DateTime _lastLoginTime;
    
    public string FirstName { get; set; }
    public string LastName { get; set; }
    
    public string Email 
    { 
        get => _email;
        set 
        {
            if (string.IsNullOrEmpty(value) || !value.Contains("@"))
                throw new ArgumentException("Invalid email format");
            _email = value;
        }
    }
    
    public string FullName => $"{FirstName} {LastName}";
    
    public bool IsActive => DateTime.Now - _lastLoginTime < TimeSpan.FromDays(30);
    
    public void Login()
    {
        _lastLoginTime = DateTime.Now;
        Console.WriteLine($"User {FullName} logged in at {_lastLoginTime}");
    }
    
    public bool CanAccessFeature(UserRole role)
    {
        // Business logic based on user role
        return role != UserRole.Guest && IsActive;
    }
}

In [ ]:
// Usage example
var user = new User { FirstName = "John", LastName = "Doe", Email = "john@example.com" };
user.Login();
Console.WriteLine($"Is active: {user.IsActive}");

### ❌ Bad Example - Trivial Class Usage

In [ ]:
// ❌ BAD: Just a data container with no validation or behavior
public class BadUser
{
    public string Name { get; set; }
    public string Email { get; set; }
    public int Age { get; set; }
}

// This is just a data bag - could be a record instead
// No validation, no business logic, no encapsulation

## Structs

### ✅ Good Example - Meaningful Struct Usage

In [ ]:
// ✅ GOOD: Money struct - immutable value type representing currency
public readonly struct Money : IEquatable<Money>, IComparable<Money>
{
    public decimal Amount { get; }
    public string Currency { get; }
    
    public Money(decimal amount, string currency)
    {
        if (amount < 0) throw new ArgumentException("Amount cannot be negative");
        if (string.IsNullOrEmpty(currency)) throw new ArgumentException("Currency is required");
        
        Amount = amount;
        Currency = currency.ToUpperInvariant();
    }
    
    // Value equality
    public bool Equals(Money other) => Amount == other.Amount && Currency == other.Currency;
    public override bool Equals(object obj) => obj is Money other && Equals(other);
    public override int GetHashCode() => HashCode.Combine(Amount, Currency);
    
    // Comparison
    public int CompareTo(Money other)
    {
        if (Currency != other.Currency) 
            throw new InvalidOperationException("Cannot compare different currencies");
        return Amount.CompareTo(other.Amount);
    }
    
    // Operators
    public static Money operator +(Money left, Money right)
    {
        if (left.Currency != right.Currency)
            throw new InvalidOperationException("Cannot add different currencies");
        return new Money(left.Amount + right.Amount, left.Currency);
    }
    
    public override string ToString() => $"{Amount:C} {Currency}";
}

In [ ]:
// Usage example
var price1 = new Money(29.99m, "USD");
var price2 = new Money(19.99m, "USD");
var total = price1 + price2;
Console.WriteLine($"Total: {total}");

### ❌ Bad Example - Trivial Struct Usage

In [ ]:
// ❌ BAD: Large mutable struct - should be a class
public struct BadPerson
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Address { get; set; }
    public string Phone { get; set; }
    public DateTime BirthDate { get; set; }
    public List<string> Hobbies { get; set; } // Reference type in struct!
}

// Problems:
// 1. Too large (structs should be small)
// 2. Mutable (structs should be immutable)
// 3. Contains reference types
// 4. No validation or meaningful behavior

## Records

### ✅ Good Example - Meaningful Record Usage

In [ ]:
// ✅ GOOD: DTOs for data transfer between layers
public record ProductDto(int Id, string Name, decimal Price, ProductCategory Category)
{
    // Computed property
    public string DisplayName => $"{Name} ({Category})"; 
    
    // Validation
    public bool IsValid => !string.IsNullOrEmpty(Name) && Price > 0;
}

// Record for API responses
public record ApiResponse<T>(bool Success, T Data, string ErrorMessage = null)
{
    public static ApiResponse<T> Ok(T data) => new(true, data);
    public static ApiResponse<T> Error(string message) => new(false, default(T), message);
}

// Record for configuration
public record DatabaseConfig(
    string ConnectionString,
    int TimeoutSeconds = 30,
    bool EnableLogging = false
);

In [ ]:
// Usage examples
var product = new ProductDto(1, "Laptop", 999.99m, ProductCategory.Electronics);
var response = ApiResponse<ProductDto>.Ok(product);
var config = new DatabaseConfig("Server=localhost;Database=MyDb", EnableLogging: true);

Console.WriteLine(product.DisplayName);
Console.WriteLine($"Product is valid: {product.IsValid}");

// Records provide value equality automatically
var product2 = product with { Price = 899.99m };
Console.WriteLine($"Same product: {product == product2}"); // False - different price

### ❌ Bad Example - Trivial Record Usage

In [ ]:
// ❌ BAD: Using record when class would be better
public record BadUserRecord(string Name, string Email)
{
    // Mutable state in record - defeats the purpose
    public List<string> LoginHistory { get; set; } = new();
    
    // Methods that modify state
    public void AddLogin(string timestamp)
    {
        LoginHistory.Add(timestamp); // Modifying state
    }
}

// This should be a class because:
// 1. It has mutable state
// 2. It has behavior that modifies state
// 3. Records are for immutable data

## Enums

### ✅ Good Example - Meaningful Enum Usage

In [ ]:
// ✅ GOOD: Enums used in business logic with meaningful values
public enum UserRole
{
    Guest = 0,
    User = 1,
    Moderator = 2,
    Admin = 3
}

public enum ProductCategory
{
    Electronics,
    Clothing,
    Books,
    Home,
    Sports
}

// Flags enum for permissions
[Flags]
public enum Permission
{
    None = 0,
    Read = 1,
    Write = 2,
    Delete = 4,
    Admin = 8,
    
    // Combined permissions
    ReadWrite = Read | Write,
    All = Read | Write | Delete | Admin
}

// Extension methods for enum
public static class UserRoleExtensions
{
    public static string GetDisplayName(this UserRole role) => role switch
    {
        UserRole.Guest => "Guest User",
        UserRole.User => "Regular User",
        UserRole.Moderator => "Content Moderator",
        UserRole.Admin => "System Administrator",
        _ => role.ToString()
    };
    
    public static bool CanModerate(this UserRole role) => 
        role >= UserRole.Moderator;
}

// Usage in business logic
public class UserService
{
    public bool CanAccessFeature(UserRole userRole, Permission requiredPermission)
    {
        var userPermissions = GetUserPermissions(userRole);
        return userPermissions.HasFlag(requiredPermission);
    }
    
    private Permission GetUserPermissions(UserRole role) => role switch
    {
        UserRole.Guest => Permission.Read,
        UserRole.User => Permission.ReadWrite,
        UserRole.Moderator => Permission.ReadWrite | Permission.Delete,
        UserRole.Admin => Permission.All,
        _ => Permission.None
    };
}

In [ ]:
// Usage examples
var role = UserRole.Moderator;
Console.WriteLine($"Role: {role.GetDisplayName()}");
Console.WriteLine($"Can moderate: {role.CanModerate()}");

var service = new UserService();
Console.WriteLine($"Can delete: {service.CanAccessFeature(role, Permission.Delete)}");

// Flags enum usage
var permissions = Permission.Read | Permission.Write;
Console.WriteLine($"Has read permission: {permissions.HasFlag(Permission.Read)}");
Console.WriteLine($"Has admin permission: {permissions.HasFlag(Permission.Admin)}");

### ❌ Bad Example - Trivial Enum Usage

In [ ]:
// ❌ BAD: Meaningless enum that's never used in logic
public enum Color
{
    Red,
    Green,
    Blue
}

// Just created to satisfy requirement, never used meaningfully:
public class SomeClass
{
    public Color SomeColor { get; set; } = Color.Red;
    
    // Color is never used in any business logic
    // No validation, no behavior based on color
    // Could be replaced with string or removed entirely
}

// Another bad example - magic numbers disguised as enum
public enum BadConstants
{
    Thing1 = 1,
    Thing2 = 2,
    Thing3 = 3
}

// No clear purpose, no business meaning

## Summary

### What Makes Usage "Meaningful":

**Classes:**
- Encapsulation with private fields
- Validation in properties
- Business logic methods
- Computed properties
- State management

**Structs:**
- Small, immutable value types
- Value equality implementation
- Mathematical or coordinate-like data
- Readonly when possible
- Operators for natural operations

**Records:**
- Data transfer objects (DTOs)
- Configuration objects
- API responses/requests
- Immutable data containers
- Value equality by default

**Enums:**
- Used in business logic decisions
- Switch expressions/statements
- Flags for combinable options
- Extension methods for behavior
- Clear business meaning

### Red Flags (Avoid These):
- Empty classes with only auto-properties
- Large mutable structs
- Records with mutable state
- Enums that are never used in logic
- Copy-pasted examples without context